In [1]:
# reloading magic
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.signal import butter, cheby1, cheby2, medfilt
from scipy.signal import lfilter, freqz
import seaborn as sns
%matplotlib qt

In [3]:
# WINDOWS
comm_path = os.path.normpath('C:/cygwin64/home/oliverlee/repos/bikesim/comm')
scripts_path = os.path.normpath('C:/cygwin64/home/oliverlee/repos/bikesim/scripts')

In [ ]:
# OSX
comm_path = os.path.normpath('/Users/oliverlee/repos/bikesim/comm')
scripts_path = os.path.normpath('/Users/oliverlee/repos/bikesim/scripts')

In [3]:
# linux
comm_path = os.path.normpath('/home/oliver/repos/bikesim/comm')
scripts_path = os.path.normpath('/home/oliver/repos/bikesim/scripts')

In [4]:
for d in (comm_path, scripts_path):
    if d not in sys.path:
        sys.path.append(d)
from parse_data import *

In [20]:
log_file = 'C:/cygwin64/home/oliverlee/repos/bikesim/comm/data/log_150618_123728_UTC_100Hz_nofilter'

In [4]:
log_file = 'C:/cygwin64/home/oliverlee/repos/bikesim/comm/data/log_150618_124057_UTC_50Hz_nofilter'

In [103]:
log_file = 'C:/cygwin64/home/oliverlee/repos/bikesim/comm/log_150622_175024_UTC'

In [3]:
log_file = '../comm/log_150618_230058_UTC'

In [5]:
log_file = None

In [6]:
# get the most recent log file in the comm_dir if not specified
if log_file is None:
    logs = [f for f in os.listdir(comm_path)
            if f.startswith('log_') and os.path.isfile(os.path.join(comm_path, f))]
    logs.sort()
    log_file = os.path.join(comm_path, logs[-1])
log_file = os.path.normpath(log_file)
print('reading from log file: {}'.format(log_file))
assert os.path.isfile(log_file)

reading from log file: /home/oliver/repos/bikesim/comm/log_150624_083310_UTC


In [15]:
sensor, actuator = parse_log(log_file)

In [8]:
fig1, ax1, hd1 = plot_timeinfo(sensor)
if actuator.shape[0] > 1:
    fig2, ax2, hd2 = plot_timeinfo(actuator, 100) # ignore dt > 100 ms
plt.show()

sensor time mean = 19.993719 s, std = 0.322871
actuator time mean = 19.737591 s, std = 2.533890


In [13]:
fig, ax1 = plt.subplots()
delta = sensor.delta*180/np.pi
ax1.plot(sensor.time, delta, 'b-')
ax1.set_xlabel('time [s]')
ax1.set_ylabel('delta [deg]', color='b')
print('delta -- mean: {:0.6f}, std: {:0.6f}'.format(np.mean(delta), np.std(delta)))

deltad = sensor.deltad*180/np.pi
ax2 = ax1.twinx()
ax2.plot(sensor.time, deltad, 'r-')
ax2.set_ylabel('deltad [deg/s]', color='r')
print('deltad -- mean: {:0.6f}, std: {:0.6f}'.format(np.mean(deltad), np.std(deltad)))

align_yaxis(ax1, 0, ax2, 0)
ax1.grid()
fig.suptitle('{} - {}'.format(sensor.filepath, 'delta, deltad'))
plt.show()

delta -- mean: 14.724906, std: 10.874523
deltad -- mean: -0.753444, std: 29.811441


In [54]:
plot_lean_steer(sensor, actuator)

(<matplotlib.figure.Figure at 0xbb09a20>,
 <matplotlib.axes._subplots.AxesSubplot at 0xbd37630>)

In [70]:
lim = axes[1].get_ylim()
orig_lim = (-20, 100)



TypeError: unorderable types: map() > int()

In [62]:
fig, axes = plot_lean_steer_yy(sensor, actuator)
print(axes[0].transData.transform((0, 0)))
print(axes[1].transData.transform((0, 0)))


(-20.0, 100.0)
(-40.0, 80.0)
[-1086.83865427   161.33333333]
[-1086.83865427   161.33333333]


In [17]:
fig, axes = plot_subplots(sensor, actuator, ('delta', 'deltad', 'torque', 'phi'))

In [16]:
plot_subplots(sensor, actuator, ('torque',))

(<matplotlib.figure.Figure at 0x8d1a080>,
 (<matplotlib.axes._subplots.AxesSubplot at 0x8ca4b70>,))

In [17]:
plot_lean_steer(sensor, actuator)

(<matplotlib.figure.Figure at 0x8c91400>,
 <matplotlib.axes._subplots.AxesSubplot at 0x8cca7b8>)

In [18]:
fig, axes = plot_singleplot(sensor, actuator, ('delta', 'deltad', 'torque', 'phi'))

In [23]:
fig, ax = plot_lean_steer(sensor, actuator)

In [15]:
a = np.ones((3, 3))
a[:,1]  = 2*a[:,1]
print(a)

[[ 1.  2.  1.]
 [ 1.  2.  1.]
 [ 1.  2.  1.]]


In [58]:
fig, ax = plt.subplots()
deltad = sensor.deltad*180/np.pi
ax.plot(sensor.time, deltad,)
ax.set_ylabel('deltad [deg/s]')
ax.set_xlabel('time [s]')
print('deltad -- mean: {:0.6f}, std: {:0.6f}'.format(np.mean(deltad), np.std(deltad)))

plt.show()

deltad -- mean: -0.351705, std: 12.126472


In [66]:
deltad = sensor.deltad * 180 / np.pi
sample_period = np.round(np.mean(sensor.dt)*100)/100
sample_rate = 1/sample_period
cutoff = 5
order = 4
rs1 = 0.2
rs2 = 10

#filters = (butter, cheby1, cheby2)
filters = (butter,)
filter_coeffs = {
    butter: butter(order, cutoff/(0.5*sample_rate), btype='low', analog=False),
    cheby1: cheby1(order, rs1, cutoff/(0.5*sample_rate), btype='low', analog=False),
    cheby2: cheby2(order, rs2, cutoff/(0.5*sample_rate), btype='low', analog=False),
}

def color_cycle(n):
    return iter(cm.Accent(np.linspace(0, 1, 5)))

def filter_data(filter_type, data):
    if filter_type == medfilt:
        return medfilt(data, 5)
    b, a = filter_coeffs[filter_type]
    return lfilter(b, a, data)

def plot_freqresp(filter_type, ax, color):
    if filter_type == medfilt:
        return
    b, a = filter_coeffs[filter_type]
    w, h = freqz(b, a, worN=8000)
    ax.plot(0.5*sample_rate*w/np.pi, np.abs(h), color=color)
    ax.plot(cutoff, 0.5*np.sqrt(2), 'ko')
    ax.axvline(cutoff, color='k')
    ax.set_xlim(0, 0.5*sample_rate)
    ax.set_xlabel('Frequency [Hz]')
    ax.grid()

def plot_filterdata(filter_type, ax, color):
    data = filter_data(filter_type, deltad)
    ax.plot(sensor.time, data, '-', color=color, linewidth=2, label=filter_type.__name__)
    
deltad_mean = np.mean(deltad)
deltad_std = np.std(deltad)

fig, (ax1, ax2) = plt.subplots(2, 1)
colors = color_cycle(len(filters) + 2)
plt.title("Lowpass Filter Frequency Response")
for f in filters:
    c = next(colors)
    plot_freqresp(f, ax1, c)
    plot_filterdata(f, ax2, c)
c = next(colors)
c[-1] = 0.5 # reduce alpha
ax2.plot(sensor.time, deltad, '--', color=c, label='deltad')
ax2.set_xlabel('Time [sec]')
ax2.set_ylabel('deltad [deg/s]')
ax2.set_xlim([sensor.time[0], sensor.time[-1]])
ax2.set_ylim([deltad_mean-deltad_std, deltad_mean+deltad_std])
ax2.grid()
plt.legend()
plt.subplots_adjust(hspace=0.35)
plt.show()



In [71]:
import collections
b, a = butter(5, 5/(0.5*100), btype='low', analog=False)
x = collections.deque(5*[0], 5)
l = [(-i, a) for i, a in enumerate(a[1:], 1)]
print(l)
print(a)

[(-1, -3.984543119612336), (-2, 6.4348670902758682), (-3, -5.2536151703522682), (-4, 2.1651329097241323), (-5, -0.35992824506355614)]
[ 1.         -3.98454312  6.43486709 -5.25361517  2.16513291 -0.35992825]
